In [ ]:
import pandas as pd
import numpy as np

In [ ]:
sample = pd.read_csv('/home/yuan/Dropbox/nerual network/predict_20min_avg_travel_time_addWeather.csv')
sample = sample.drop(sample.columns[0], axis=1)
answer = pd.read_csv('/home/yuan/Dataset/kdd_2017/submission_sample_travelTime.csv')

In [ ]:
sample

In [ ]:
sample['extr_time'] = pd.to_datetime(sample['time_window'].apply(lambda x: x.strip('[)').split(',')[0]))
sample['hour'] = sample['extr_time'].dt.hour
sample['min'] = sample['extr_time'].dt.minute
sample['week'] = sample['extr_time'].dt.weekday


In [ ]:
sample['trip_type'] = float('NaN')
sample['trip_type'][(sample['intersection_id']=='A') & (sample['tollgate_id']==2)] = 0
sample['trip_type'][(sample['intersection_id']=='A') & (sample['tollgate_id']==3)] = 1
sample['trip_type'][(sample['intersection_id']=='B') & (sample['tollgate_id']==1)] = 2
sample['trip_type'][(sample['intersection_id']=='B') & (sample['tollgate_id']==3)] = 3
sample['trip_type'][(sample['intersection_id']=='C') & (sample['tollgate_id']==1)] = 4
sample['trip_type'][(sample['intersection_id']=='C') & (sample['tollgate_id']==3)] = 5

In [ ]:
hr_min = []
for row in range(len(sample)):
    hr_min.append(str(sample['hour'][row])+'-'+str(sample['min'][row]))
    
sample['hr_min'] = hr_min    

In [ ]:
feature_test = sample[['valuePrecipitation','wind_speed','temperature','rel_humidity','hour','week','trip_type','min']].values

In [ ]:
type(feature_test)

In [ ]:
test = pd.read_csv('/home/yuan/Dropbox/nerual network/test1_20min_avg_travel_time_addWeather.csv')
test = test.drop(test.columns[0], axis=1)

In [ ]:
test['extr_time'] = pd.to_datetime(test['time_window'].apply(lambda x: x.strip('[)').split(',')[0]))
test['hour'] = test['extr_time'].dt.hour
test['week'] = test['extr_time'].dt.weekday

In [ ]:
train = pd.read_csv('/home/yuan/Dropbox/nerual network/training_20min_avg_travel_time_addWeather.csv')
train = train.drop(train.columns[0], axis=1)

In [ ]:
train

In [ ]:
train['extr_time'] = pd.to_datetime(train['time_window'].apply(lambda x: x.strip('[)').split(',')[0]))
train['hour'] = train['extr_time'].dt.hour
train['min'] = train['extr_time'].dt.minute
train['week'] = train['extr_time'].dt.weekday


In [ ]:
hr_min = []
for row in range(len(train)):
    hr_min.append(str(train['hour'][row])+'-'+str(train['min'][row]))
    
train['hr_min'] = hr_min    

In [ ]:
train['trip_type'] = float('NaN')
train['trip_type'][(train['intersection_id']=='A') & (train['tollgate_id']==2)] = 0
train['trip_type'][(train['intersection_id']=='A') & (train['tollgate_id']==3)] = 1
train['trip_type'][(train['intersection_id']=='B') & (train['tollgate_id']==1)] = 2
train['trip_type'][(train['intersection_id']=='B') & (train['tollgate_id']==3)] = 3
train['trip_type'][(train['intersection_id']=='C') & (train['tollgate_id']==1)] = 4
train['trip_type'][(train['intersection_id']=='C') & (train['tollgate_id']==3)] = 5

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split

In [ ]:
import xgboost as xgb

In [ ]:
xgb_clf = xgb.XGBClassifier()

In [ ]:
feature = train[['valuePrecipitation','wind_speed','temperature','rel_humidity','hour','week','trip_type','min']].values
target = train['avg_travel_time'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=42)

In [ ]:
%%time
forest = xgb_clf.fit(X_train,y_train)


In [ ]:
pred_forest = forest.predict(X_test)
error = abs(pred_forest - y_test) / y_test
error.mean()

In [ ]:
feature.shape

In [ ]:
target.shape

In [ ]:
train['avg_travel_time'][train['avg_travel_time']>150]

In [ ]:
xgb_clf.fit(feature, target)

In [ ]:
clf = RandomForestClassifier()
my_forest = clf.fit(feature, target)

In [ ]:
ans_label = my_forest.predict(feature_test)

In [ ]:
answer['avg_travel_time'] = ans_label

In [ ]:
answer.to_csv('first_submission.csv',index=None)

In [ ]:
my_forest.feature_importances_

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=42)
forest = clf.fit(X_train,y_train)
pred_forest = forest.predict(X_test)
error = abs(pred_forest - y_test) / y_test
error.mean()

In [ ]:
cross_val_score(clf, feature, target, cv=3)

In [ ]:
import xgboost as xgb


In [ ]:
xgb_clf = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)

In [ ]:
gbm = xgb_clf.fit(feature, target)